In [1]:
# import necessary library
import pandas as pd

In [2]:
# fetch the file using a link; unix core
!curl -O https://ftp.ncbi.nih.gov/genomes/refseq/vertebrate_mammalian/Homo_sapiens/latest_assembly_versions/GCF_000001405.39_GRCh38.p13/GCF_000001405.39_GRCh38.p13_genomic.fna.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  920M  100  920M    0     0  57.4M      0  0:00:16  0:00:16 --:--:-- 54.0M


In [3]:
!gunzip /content/GCF_000001405.39_GRCh38.p13_genomic.fna.gz

In [4]:

# install the pyfaidx 
!pip install pyfaidx

  Created wheel for pyfaidx: filename=pyfaidx-0.5.9.5-cp37-none-any.whl size=25142 sha256=376f1b49f758ad335423affc4bbcc8469170e009299a227a4e4c43565cb9e757
  Stored in directory: /root/.cache/pip/wheels/d8/31/5f/8053c426a420cc407492252723f20e9a9c7e717909d7e08a9e
Successfully built pyfaidx


In [5]:
# import the pyfaidx
from pyfaidx import Fasta

In [6]:
# read all fasta chromosoms and convert it to upper letters
genes = Fasta('/content/GCF_000001405.39_GRCh38.p13_genomic.fna', sequence_always_upper=True)

In [7]:
# import the key of genes 
Fasta_Keys = genes.keys()

In [8]:
# extract the fasta keys from the human fasta file 
df_Fasta_Keys = pd.DataFrame(Fasta_Keys,columns=['Fasta_Keys'])

In [10]:
# Read the file of cluster 1
DF_class1 = pd.read_csv('/content/ensemble_genes_IDs_updated_df3_class2.csv')
DF_class1.head()

,chrom,chromStart,chromEnd,strand,Gene_ID
0,NC_000001.11,1020102,1056119,+,ENSG00000188157
1,NC_000001.11,1216908,1232005,-,ENSG00000078808
2,NC_000001.11,1352689,1363541,-,ENSG00000162576
3,NC_000001.11,1385711,1399336,-,ENSG00000221978
4,NC_000001.11,1418420,1422471,-,ENSG00000235098


In [11]:
# chromosome code and a subsequence unclutide from 944203 to 959256 
genes['NC_000001.11'][944203:959256]

>NC_000001.11:944204-959256
CAACACAATGGCCCTGCCTCCCACCGCTTTATTTCTTTCGGTTTCGGATGCAAAACAAAAAATTTTAAAAGAAAATGTGACTTCAAAGGAAAGGAACAAATTTTCAAAGACTTGGGGGAGTGAAGGCAGAGCCTGGTGCAGATGGACGAGGTCTGCAGACGGAGGGCAGAGGTGGTGGAAGGGGCCAGGGGCCTGCAGGCCTCCCCCTGGAACTGGGACTGGTCTCGGTCTGCTGACGTCAGGGTCAGCTCCCCCGCGGAGCTGACTTCAGCAGCCCACAGCTGTGGGGCTTCAGCAGCCACACCAGCCCAGCCCAGCCCAGCTCTCGATACGTTTGGTCTTTCATGCTGAAAAATAAATAATAAAGCCTGTCCCGTGTCTACTGCCTCCCCCAACTGCACAGACGCCAGCCTCTAGGCCTGACTGCCAGGGAGGTGGAAACACTGGCCACCAGCCCGGCAGCCCCTACAGGCCCCCCAGATGGGCTGCCTCAGTCGTCCTCTGAGAGCTGCAGATCCTCCAGCTCGTCCTCCGGCCCCTGGGCCAGCTGCTGCAGCTCCCCAGGGGCCAGCCCCGCCTCTGCGTCTGGGTCTCCATCTGCGGGGAGAGATGGAGGCTACATAAATTTTGCTTTATCAGGAAGAAGCCAGCCTTAGAGGTTACTCATCACTAATTAATCACGGCACTAATTAATTTATCCCTGTTGCTGGCTGCCAGAGAACAGAGCATTTGGCCTGGCCTTCCCAGGGAGGGAAAAGCCTGGCCCAGAGCCCCACGCCCCCCGCCCACGTGGCTCTGCCCTCCCGCCAGATGGGCTCACAGGGCCACACCCTCTCACCCCAAGACCATTCACCCTCCGAGTTGCTGCTGTCCTCCTCGCCCTCCTCCTCGTCCTCTTCATCGTCTTCCACCCCATGCCGAGTGCTCAGGGGCCTCAGTATCCCTGAGGAACAAGAAGCAGAGTCCATATGA

In [12]:
# Chromosome 500 in chrome column
genes[DF_class1.iloc[500]['chrom ']]

FastaRecord("NC_000003.12")

In [13]:
# Fetch the sequence based on the start and the end of genes, then replace the gene ensembl with the chromosome
new_fasta = []
for gen in range(len(df_Fasta_Keys)):
  #apply a function based on the strand
  #inverse and complement
  if DF_class1.iloc[gen]['strand'] == '-':
    # equation: chromEnd + 1000 up to chromEnd
    #starting nuectuotide
    start = DF_class1.iloc[gen]['chromEnd']
    #ending nuectuotide
    end = start + 1000 
    seq = -genes[DF_class1.iloc[gen]['chrom ']][start:end]
    new_fasta.append('>%s\n%s' % (DF_class1.iloc[gen]['Gene_ID'], seq))
  # Positive strand: takes the 1000 upstream from ChromStart in the same subsequence
  else:
    #starting nuectuotide
    # chromStart - 1000 
    start = (DF_class1.iloc[gen]['chromStart'])-1000
    #ending nuectuotide
    end = DF_class1.iloc[gen]['chromStart']
    seq = genes[DF_class1.iloc[gen]['chrom ']][start:end]
    new_fasta.append('>%s\n%s' % (DF_class1.iloc[gen]['Gene_ID'], seq))

In [14]:
# saving the result of mapped ensembl with the selected portion to a fasta file
with open('genes_DF_Class2.fasta', 'w') as f:
    f.write('\n'.join(new_fasta))